In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import  KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [45]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [46]:
import re
import pandas as pd
import numpy as np
import os

url = os.chdir(r"/content/gdrive/MyDrive/Data Science/данные")
os.getcwd()

'/content/gdrive/MyDrive/Data Science/данные'

In [ ]:
files = os.listdir(url)

k = ['train_bg', 'train_cs', 'train_da', 'train_de', 'train_el', 
     'train_es', 'train_et', 'train_fi', 'train_fr', 'train_hu', 
     'train_it', 'train_lt', 'train_lv', 'train_nl', 'train_pl', 
     'train_pt', 'train_ro', 'train_sk', 'train_sl', 'train_sv']

lang = {'train_bg':[], 'train_cs':[], 'train_da':[], 'train_de':[], 'train_el':[], 
        'train_es':[], 'train_et':[], 'train_fi':[], 'train_fr':[], 'train_hu':[], 
        'train_it':[], 'train_lt':[], 'train_lv':[], 'train_nl':[], 'train_pl':[], 
        'train_pt':[], 'train_ro':[], 'train_sk':[], 'train_sl':[], 'train_sv':[]}

i = 0
for file in files:
    with open(file, "r", encoding='utf-8') as f:
        for line in f.readlines():
            lang[k[i]].append(line)

    i += 1

In [74]:
train = pd.DataFrame(columns=['text', 'language'])

for l in lang:
    if re.findall('train', l):
        tmp = pd.DataFrame()
        tmp['text'] = lang[l]
        tmp['language'] = l.split('train_')[1]
        tmp = tmp[:7500]
        train = pd.concat([train, tmp], ignore_index=True)

In [75]:
train

,text,language
0,Внасяне на документи: вж. протоколи\n,bg
1,Дневен ред на следващото заседание: вж. проток...,bg
2,Dichiaro interrotta la sessione del Parlamento...,bg
3,Искане за защита на парламентарен имунитет: вж...,bg
4,Изпращане на текстове на споразумения от Съвет...,bg
...,...,...
149995,"En nogmaals, dit is geen pleidooi voor regelge...",sv
149996,Wel dient meer doorzichtigheid te worden gegev...,sv
149997,Ik kan dat niet omdat de mededeling een typisc...,sv
149998,Of om de verstikkende regelgeving die momentee...,sv


In [76]:
print(train.info())
print('-------------------------------------------------------')
print(train.isna().sum())
print('-------------------------------------------------------')
print(train['language'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   text      150000 non-null  object
 1   language  150000 non-null  object
dtypes: object(2)
memory usage: 2.3+ MB
None
-------------------------------------------------------
text        0
language    0
dtype: int64
-------------------------------------------------------
fr    7500
de    7500
sl    7500
et    7500
es    7500
el    7500
hu    7500
bg    7500
nl    7500
it    7500
sk    7500
ro    7500
fi    7500
da    7500
lv    7500
lt    7500
cs    7500
sv    7500
pl    7500
pt    7500
Name: language, dtype: int64


In [77]:
train['text_01'] = train['text'].str.replace('"', "")
train['text_01'] = train['text_01'].str.replace('…', "")

signs = ['?', ',', ':', '!', '.', ';', '«', '»', '(', ')', '*', '+', '-', '^', '|', '=', "’", "‘",  '”', '—', "/", "'", "¿", '¡', '%', '·']
for punct_sign in signs:
    train['text_01'] = train['text_01'].str.replace(punct_sign, ' ')

train['text_01'] = train['text_01'].str.replace("\r", " ")
train['text_01'] = train['text_01'].str.replace("\n", " ")

train['text_01'] = train['text_01'].str.replace('  ', " ")
train['text_01'] = train['text_01'].str.replace('  ', " ")

In [78]:
train['text_02'] = train['text_01'].str.replace('\d+', '')
train['text_02'] = train['text_02'].str.lower()

In [79]:
train.head()

,text,language,text_01,text_02
0,Внасяне на документи: вж. протоколи\n,bg,Внасяне на документи вж протоколи,внасяне на документи вж протоколи
1,Дневен ред на следващото заседание: вж. проток...,bg,Дневен ред на следващото заседание вж протоколи,дневен ред на следващото заседание вж протоколи
2,Dichiaro interrotta la sessione del Parlamento...,bg,Dichiaro interrotta la sessione del Parlamento...,dichiaro interrotta la sessione del parlamento...
3,Искане за защита на парламентарен имунитет: вж...,bg,Искане за защита на парламентарен имунитет вж ...,искане за защита на парламентарен имунитет вж ...
4,Изпращане на текстове на споразумения от Съвет...,bg,Изпращане на текстове на споразумения от Съвет...,изпращане на текстове на споразумения от съвет...


In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [81]:
languages = ['bg', 'cs', 'da', 'de', 'el', 'es', 'et', 'fi', 'fr', 'hu', 'it', 'lt', 'lv', 'nl', 'pl', 'pt', 'ro', 'sk', 'sl', 'sv']

tf_idf = []
features = []

ft = []
lng = []

for i in languages:

    df_l = train[train['language'] == i]
    
    tfidf = TfidfVectorizer(ngram_range=(2,3), analyzer='char', sublinear_tf=True, max_features=300)

    tfidf_i = tfidf.fit_transform(df_l['text_02']).toarray()
    feature_names = tfidf.get_feature_names()

    l = [i] * 7500
    lng += l

    tf_idf.append(tfidf_i)
    features.append(feature_names)
    ft += feature_names

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is d

In [82]:
# for i in range(20):
df1 = pd.DataFrame(columns=features, data=tf_idf[0])

In [83]:
for i in ft:
    i = i.replace(' ', '')

In [84]:
for i in range(len(ft)):
    s = ft[i]
    if len(s) == 1:
        ft[i] = int(str(ord(s)))
    elif len(s) == 2:
        ft[i] = int(str(ord(s[0])) + str(ord(s[1])))
    elif len(s) == 3:
        ft[i] = int(str(ord(s[0])) + str(ord(s[1])) + str(ord(s[2])))


In [85]:
df = pd.DataFrame(columns=['Data', 'Language'])

l = []
for i in range(20):
    w = [languages[i]] * 300
    l += w

df['Data'] = ft
df['Language'] = l
df

,Data,Language
0,321072,bg
1,321073,bg
2,321074,bg
3,32107432,bg
4,3210741098,bg
...,...,...
5995,122101,sv
5996,12210132,sv
5997,122105,sv
5998,122105106,sv


In [86]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['Language'] = labelencoder.fit_transform(df['Language'])

In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Data      6000 non-null   int64
 1   Language  6000 non-null   int64
dtypes: int64(2)
memory usage: 93.9 KB


In [88]:
X = df['Data']
y = df['Language']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/3, random_state = 42)

X_train = X_train.values.reshape(-1, 1)
y_train = y_train.values.reshape(-1, 1)
X_test = X_test.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

In [90]:
# KNeighborsClassifier
p_knn = KNeighborsClassifier()
p_knn.fit(X_train, y_train)
p_knn_predict = p_knn.predict(X_test)

print(classification_report(y_test,p_knn_predict))

              precision    recall  f1-score   support

           0       0.90      0.95      0.92       116
           1       0.04      0.12      0.06       115
           2       0.98      0.96      0.97       107
           3       0.03      0.10      0.05       102
           4       0.02      0.06      0.03       100
           5       0.04      0.06      0.04       104
           6       0.04      0.07      0.05        88
           7       0.03      0.04      0.04        98
           8       0.02      0.01      0.01        94
           9       0.10      0.07      0.08        98
          10       0.09      0.04      0.06       100
          11       0.05      0.02      0.03        94
          12       0.19      0.08      0.11       116
          13       0.06      0.03      0.04        77
          14       0.17      0.02      0.04       100
          15       0.06      0.01      0.02       110
          16       0.00      0.00      0.00       103
          17       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [137]:
from google.colab import files
uploaded = files.upload()

Saving test_data.csv to test_data.csv


In [138]:
test_data = pd.read_csv('test_data.csv')
test_data

,index,text
0,0,szállítanak
1,1,pracovni
2,2,fašisti
3,3,passer
4,4,culpabilização
...,...,...
8515,8515,middags
8516,8516,tunnetega
8517,8517,tocassem
8518,8518,urale


In [139]:
test_data['text'] = test_data['text'].str.replace('"', "")
test_data['text'] = test_data['text'].str.replace('…', "")

signs = ['?', ',', ':', '!', '.', ';', '«', '»', '(', ')', '*', '+', '-', '^', '|', '=', "’", "‘",  '”', '—', "/", "'", "¿", '¡', '%', '·']
for punct_sign in signs:
    test_data['text'] = test_data['text'].str.replace(punct_sign, ' ')

test_data['text'] = test_data['text'].str.replace("\r", " ")
test_data['text'] = test_data['text'].str.replace("\n", " ")

test_data['text'] = test_data['text'].str.replace('  ', " ")
test_data['text'] = test_data['text'].str.replace('  ', " ")

In [140]:
test_data['text'] = test_data['text'].str.replace('\d+', '')
test_data['text'] = test_data['text'].str.lower()

In [104]:
def ngrams(word):
    grams_1 = []
    grams_2 = []
    grams_3 = []

    for i in range(len(word)):
        ng = word[i]
        if ng not in grams_1:
            grams_1.append(ng)

    for i in range(len(word)-1):
        ng = word[i] + word[i+1]
        if ng not in grams_2:
            grams_2.append(ng)

    for i in range(len(word)-2):
        ng = word[i] + word[i+1] + word[i+2]
        if ng not in grams_3:
            grams_3.append(ng)

    n = grams_1 + grams_2 + grams_3

    for i in range(len(n)):
        s = n[i]
        if len(s) == 1:
            n[i] = int(str(ord(s)))
        elif len(s) == 2:
            n[i] = int(str(ord(s[0])) + str(ord(s[1])))
        elif len(s) == 3:
            n[i] = int(str(ord(s[0])) + str(ord(s[1])) + str(ord(s[2])))
    return n

In [142]:
result = []

In [144]:
for i in test_data['text']:
    ngrams_word = ngrams(i)
    df = pd.DataFrame(columns=['Data'], data = ngrams_word)

    w_predict = p_knn.predict(df)

    res = pd.DataFrame(columns=['r'], data = w_predict)

    result.append(res.value_counts().max())

Выходные данные были обрезаны до нескольких последних строк (5000).
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7

In [145]:
res = []
for i in result:
    if i > 19:
        res.append(languages[19])
    else: 
        res.append(languages[i])

In [146]:
test_data['lang_samp'] = res
test_data['lang_samp'].value_counts()

hu    1306
fr    1302
fi    1180
it    1104
lt     862
et     773
lv     535
nl     346
es     260
pl     206
sv     180
pt     155
ro      92
sk      77
el      72
sl      40
de      24
da       6
Name: lang_samp, dtype: int64

In [148]:
test_data.drop(columns=['text'], axis = 1).to_csv("lang_samp_dolzhenko.csv", index=False)
files.download("lang_samp_dolzhenko.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>